## 실습해보기

### CNN 구조 확인

In [1]:
import torch
import torch.nn as nn

In [2]:
inputs = torch.Tensor(1, 1, 28, 28) #(batch_size, channel, weight, width)
print(inputs.shape)

torch.Size([1, 1, 28, 28])


In [3]:
# Layer1 설정
conv1 = nn.Conv2d(1, 32, 3, padding = 1)
pool = nn.MaxPool2d(2) # Layer에도 사용

# Layer2 설정
conv2 = nn.Conv2d(32, 64, 3, padding = 1)

In [4]:
# Layer1 통과
out = conv1(inputs)
out.shape

torch.Size([1, 32, 28, 28])

In [5]:
# Max pooling
out = pool(out)
out.shape

torch.Size([1, 32, 14, 14])

In [6]:
# Layer2 통과
out = conv2(out)
out.shape

torch.Size([1, 64, 14, 14])

In [7]:
# Max pooling
out = pool(out)
out.shape

torch.Size([1, 64, 7, 7])

In [8]:
# view, FC 설정 후 size 확인
out = out.view(out.size(0), -1) #out.size(0) = batch_size
out.shape

torch.Size([1, 3136])

In [9]:
fc = nn.Linear(3136,10)
out = fc(out)
out

tensor([[ 0.0229,  0.0667,  0.0287,  0.0806, -0.0546,  0.0962,  0.1502, -0.0794,
         -0.0162, -0.0836]], grad_fn=<AddmmBackward>)

In [10]:
out.shape

torch.Size([1, 10])

### 본격적으로 모델링!

In [11]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [13]:
# train에 사용되는 params 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [14]:
# 데이터셋 가져오기
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                         train = True,
                         transform = transforms.ToTensor(),
                         download = True)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                        train = False,
                        transform = transforms.ToTensor(),
                        download = True)

In [15]:
# (학습에 쓰기 편하게) loader 만들기
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size = batch_size,
                                         shuffle = True,
                                         drop_last = True)

In [16]:
# CNN Model (2 Conv layers)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size =3, stride = 1, padding = 1),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.fc= nn.Linear(7 * 7 * 64, 10, bias = True)
        nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) #Flatten them for FC
        out = self.fc(out)
        
        return out

In [17]:
#instantiate CNN model
model = CNN().to(device)

In [18]:
criterion = nn.CrossEntropyLoss().to(device) # Softmax is internally computed
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [19]:
# 모델 학습 및 loss check(Criterion의 output)
total_batch = len(data_loader)
print('Learning started. It takes sometime.')

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoch: {:>4}] cost {:>.9}'.format(epoch + 1, avg_cost))
    
print("Learning Finished!")

Learning started. It takes sometime.
[Epoch:    1] cost 0.223989323
[Epoch:    2] cost 0.0622294322
[Epoch:    3] cost 0.0449114479
[Epoch:    4] cost 0.0355332792
[Epoch:    5] cost 0.0290651843
[Epoch:    6] cost 0.024860831
[Epoch:    7] cost 0.0207230989
[Epoch:    8] cost 0.0181342494
[Epoch:    9] cost 0.0151020652
[Epoch:   10] cost 0.0125892395
[Epoch:   11] cost 0.010402537
[Epoch:   12] cost 0.0100278081
[Epoch:   13] cost 0.00808638241
[Epoch:   14] cost 0.0080219917
[Epoch:   15] cost 0.00593501842
Learning Finished!


In [20]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy : ', accuracy.item())

/Users/jominju/opt/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/jominju/opt/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy :  0.9850000143051147


## Q1

In [21]:
image = [[2,0,2,1,3,2],
         [0,2,0,2,2,2],
         [1,0,1,3,1,1],
         [0,0,1,1,1,0],
         [0,1,3,4,1,0],
         [0,1,0,0,5,2]]

filt = [[1,0,0],
        [0,1,0],
        [0,0,1]]

In [22]:
image

[[2, 0, 2, 1, 3, 2],
 [0, 2, 0, 2, 2, 2],
 [1, 0, 1, 3, 1, 1],
 [0, 0, 1, 1, 1, 0],
 [0, 1, 3, 4, 1, 0],
 [0, 1, 0, 0, 5, 2]]

In [23]:
filt

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]

Zero padding

In [24]:
# zero padding 크기 1
zero_padded = []
padded_len = len(image) + 2

for i in range(padded_len):
    zero_padded.append([0] * padded_len)
    if i != 0 and i != padded_len - 1:
        zero_padded[i][1:-1] = image[i-1]

In [25]:
zero_padded

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 2, 0, 2, 1, 3, 2, 0],
 [0, 0, 2, 0, 2, 2, 2, 0],
 [0, 1, 0, 1, 3, 1, 1, 0],
 [0, 0, 0, 1, 1, 1, 0, 0],
 [0, 0, 1, 3, 4, 1, 0, 0],
 [0, 0, 1, 0, 0, 5, 2, 0],
 [0, 0, 0, 0, 0, 0, 0, 0]]

Stride (1,1) filter

In [26]:
filtered = []

for i in range(6):
    new_array = []
    for j in range(6):
        val = zero_padded[i][j] + zero_padded[i+1][j+1] + zero_padded[i+2][j+2]
        new_array.append(val)
    filtered.append(new_array)

In [27]:
filtered

[[4, 0, 4, 3, 5, 2],
 [0, 5, 3, 5, 4, 5],
 [1, 1, 4, 4, 3, 3],
 [1, 4, 5, 3, 4, 1],
 [1, 1, 3, 10, 4, 1],
 [0, 1, 1, 3, 9, 3]]

Maxpooling (2,2)

In [28]:
maxpooling = []

for i in range(3):
    new_array = []
    for j in range(3):
        val = max(filtered[2 * i][2 * j], filtered[2 * i + 1][2 * j],
                 filtered[2 * i][2 * j + 1], filtered[2 * i + 1][2 * j + 1])
        new_array.append(val)
    maxpooling.append(new_array)

In [29]:
maxpooling

[[5, 5, 5], [4, 5, 4], [1, 10, 9]]

이렇게 생겼습니당

## Q2

In [39]:
class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.keep_prob = 0.8
        
        self.layer1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size =3, stride = 1, padding = 1),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer3 = nn.Sequential(nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1))
        
        self.fc1 = nn.Linear(4 * 4 * 128, 625, bias = True)
        nn.init.xavier_uniform_(self.fc1.weight)
        
        self.layer4 = nn.Sequential(self.fc1,
                                   nn.ReLU(),
                                   nn.Dropout(p = 1 - self.keep_prob))
        
        self.fc2 = nn.Linear(625, 10, bias = True)
        nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1) #Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        
        return out

In [40]:
def train(model):
    # 모델 학습 및 loss check(Criterion의 output)
    total_batch = len(data_loader)
    print('Learning started. It takes sometime.')

    for epoch in range(training_epochs):
        avg_cost = 0

        for X, Y in data_loader:
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad()
            hypothesis = model(X)
            cost = criterion(hypothesis, Y)
            cost.backward()
            optimizer.step()

            avg_cost += cost / total_batch

        print('[Epoch: {:>4}] cost {:>.9}'.format(epoch + 1, avg_cost))

    print("Learning Finished!")

In [41]:
def test(model):
    with torch.no_grad():
        X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
        Y_test = mnist_test.test_labels.to(device)

        prediction = model(X_test)
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        accuracy = correct_prediction.float().mean()
        print('Accuracy : ', accuracy.item())

In [42]:
model2 = CNN2().to(device)

In [44]:
criterion = nn.CrossEntropyLoss().to(device) # Softmax is internally computed
optimizer = torch.optim.Adam(model2.parameters(), lr = learning_rate)

In [45]:
train(model2)

Learning started. It takes sometime.
[Epoch:    1] cost 0.173473179
[Epoch:    2] cost 0.0432489552
[Epoch:    3] cost 0.0294852089
[Epoch:    4] cost 0.0255112536
[Epoch:    5] cost 0.0191292204
[Epoch:    6] cost 0.0153724179
[Epoch:    7] cost 0.0117597282
[Epoch:    8] cost 0.011674555
[Epoch:    9] cost 0.0108470572
[Epoch:   10] cost 0.00967290346
[Epoch:   11] cost 0.00749675836
[Epoch:   12] cost 0.00723878806
[Epoch:   13] cost 0.00696195802
[Epoch:   14] cost 0.00571974646
[Epoch:   15] cost 0.00615150062
Learning Finished!


In [46]:
test(model2)

Accuracy :  0.9850999712944031


In [53]:
class CNN3(nn.Module):
    def __init__(self):
        super(CNN3, self).__init__()
        self.keep_prob = 0.8
        
        self.layer1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size =4, stride = 1, padding = 1),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer3 = nn.Sequential(nn.Conv2d(64, 128, kernel_size = 3, stride = 2, padding = 1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1))
        
        self.fc1 = nn.Linear(2 * 2 * 128, 625, bias = True)
        nn.init.xavier_uniform_(self.fc1.weight)
        
        self.layer4 = nn.Sequential(self.fc1,
                                   nn.ReLU(),
                                   nn.Dropout(p = 1 - self.keep_prob))
        
        self.fc2 = nn.Linear(625, 10, bias = True)
        nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1) #Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        
        return out

In [54]:
model3 = CNN3().to(device)
criterion = nn.CrossEntropyLoss().to(device) # Softmax is internally computed
optimizer = torch.optim.Adam(model3.parameters(), lr = learning_rate)

In [55]:
train(model3)

Learning started. It takes sometime.
[Epoch:    1] cost 0.222253785
[Epoch:    2] cost 0.0570721962
[Epoch:    3] cost 0.0388892405
[Epoch:    4] cost 0.0316711552
[Epoch:    5] cost 0.0248664636
[Epoch:    6] cost 0.0201338008
[Epoch:    7] cost 0.0182972196
[Epoch:    8] cost 0.0135526862
[Epoch:    9] cost 0.0137856705
[Epoch:   10] cost 0.0110776527
[Epoch:   11] cost 0.0112223439
[Epoch:   12] cost 0.0102123776
[Epoch:   13] cost 0.00889295526
[Epoch:   14] cost 0.00852344465
[Epoch:   15] cost 0.00648356695
Learning Finished!


In [56]:
test(model3)

Accuracy :  0.987500011920929


- Layer3 이후 크기 : "After Layer3 Shape :  torch.Size([100, 128, 2, 2])"
- cost는 조금 더 증가했고, accuracy도 조금 더 증가했다